In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#데이터 초기화
def load_train():
    train = pd.read_csv("data/bike-sharing-demand/train.csv", parse_dates= ["datetime"])
    train["year"] = train.datetime.dt.year
    train["month"] = train.datetime.dt.month
    train["day"] = train.datetime.dt.day
    train["hour"] = train.datetime.dt.hour
    train["dayofweek"] = train["datetime"].dt.weekday
    return train

### windspeed열 평균으로 대체

In [3]:
train = load_train()

In [4]:
train["windspeed"][train["windspeed"] == 0] = train["windspeed"].mean()


<ipython-input-4-5fadbe5f4026>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["windspeed"][train["windspeed"] == 0] = train["windspeed"].mean()


In [5]:
(train["windspeed"] == 0).sum()

0

In [6]:
train.to_csv("train_mean.csv")
train

datetime  season  holiday  workingday  weather   temp  \
0     2011-01-01 00:00:00       1        0           0        1   9.84   
1     2011-01-01 01:00:00       1        0           0        1   9.02   
2     2011-01-01 02:00:00       1        0           0        1   9.02   
3     2011-01-01 03:00:00       1        0           0        1   9.84   
4     2011-01-01 04:00:00       1        0           0        1   9.84   
...                   ...     ...      ...         ...      ...    ...   
10881 2012-12-19 19:00:00       4        0           1        1  15.58   
10882 2012-12-19 20:00:00       4        0           1        1  14.76   
10883 2012-12-19 21:00:00       4        0           1        1  13.94   
10884 2012-12-19 22:00:00       4        0           1        1  13.94   
10885 2012-12-19 23:00:00       4        0           1        1  13.12   

        atemp  humidity  windspeed  casual  registered  count  year  month  \
0      14.395        81  12.799395       3          13     16  2011      1   
1      13.635        80  12.799395       8          32     40  2011      1   
2      13.635        80  12.799395       5          27     32  2011      1   
3      14.395        75  12.799395       3          10     13  2011      1   
4      14.395        75  12.799395       0           1      1  2011      1   
...       ...       ...        ...     ...         ...    ...   ...    ...   
10881  19.695        50  26.002700       7         329    336  2012     12   
10882  17.425        57  15.001300      10         231    241  2012     12   
10883  15.910        61  15.001300       4         164    168  2012     12   
10884  17.425        61   6.003200      12         117    129  2012     12   
10885  16.665        66   8.998100       4          84     88  2012     12   

       day  hour  dayofweek  
0        1     0          5  
1        1     1          5  
2        1     2          5  
3        1     3          5  
4        1     4          5  
...    ...   ...        ...  
10881   19    19          2  
10882   19    20          2  
10883   19    21          2  
10884   19    22          2  
10885   19    23          2  

[10886 rows x 17 columns]

### windspeed열 최소값/최대값으로 대체

In [7]:
#최대값
train = load_train()
train.windspeed[train["windspeed"] == 0] = train["windspeed"].max()
train.to_csv("train_max.csv")

<ipython-input-7-6ef15bf08353>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.windspeed[train["windspeed"] == 0] = train["windspeed"].max()


In [8]:
#최대값
train = load_train()
train["windspeed"][train["windspeed"] == 0] = train["windspeed"][train["windspeed"] != 0].min()
train.to_csv("train_min.csv")

<ipython-input-8-5c026a9c1485>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["windspeed"][train["windspeed"] == 0] = train["windspeed"][train["windspeed"] != 0].min()


### windspeed열 weather(season,...) 그룹화 -> 그룹별 windspeed평균 => 값으로 대체

In [9]:
# weather
train = load_train()
season = train.groupby("weather")["windspeed"].mean()
train["windspeed"][train["windspeed"] == 0] = train["weather"].map(season)
train.to_csv("train_weather.csv")

<ipython-input-9-540656f9fe77>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["windspeed"][train["windspeed"] == 0] = train["weather"].map(season)


In [10]:
# season
train = load_train()
season = train.groupby("season")["windspeed"].mean()
train["windspeed"][train["windspeed"] == 0] = train["season"].map(season)
train.to_csv("train_season.csv")

<ipython-input-10-75c409f8bb5f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["windspeed"][train["windspeed"] == 0] = train["season"].map(season)


### windspeed값이 0인 행 데이터와 가장 유사한 행 data를 n개 추출 -> n개 데이터의 windspeed 평균 => 값으로 대체

In [11]:
train = load_train()
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'dayofweek'],
      dtype='object')

In [12]:
# 상관 높은 변수들 추출
newtrain = train[["windspeed","temp","atemp","humidity"]]
newtrain

windspeed   temp   atemp  humidity
0         0.0000   9.84  14.395        81
1         0.0000   9.02  13.635        80
2         0.0000   9.02  13.635        80
3         0.0000   9.84  14.395        75
4         0.0000   9.84  14.395        75
...          ...    ...     ...       ...
10881    26.0027  15.58  19.695        50
10882    15.0013  14.76  17.425        57
10883    15.0013  13.94  15.910        61
10884     6.0032  13.94  17.425        61
10885     8.9981  13.12  16.665        66

[10886 rows x 4 columns]

In [13]:
#코사인 유사도
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
       return dot(A[1:], B[1:])/(norm(A[1:])*norm(B[1:]))

In [14]:
wind0 = np.array(newtrain[newtrain["windspeed"] == 0])
wind0
wind1 = np.array(newtrain[newtrain["windspeed"] != 0])
wind1

array([[ 6.0032,  9.84  , 12.88  , 75.    ],
       [16.9979, 15.58  , 19.695 , 76.    ],
       [19.0012, 14.76  , 16.665 , 81.    ],
       ...,
       [15.0013, 13.94  , 15.91  , 61.    ],
       [ 6.0032, 13.94  , 17.425 , 61.    ],
       [ 8.9981, 13.12  , 16.665 , 66.    ]])

In [15]:
# windspeed 가 0인 행과 거리가 가까운 10개 행의 windspeed 평균으로 보간하는 함수
def cos_mean(idx0):
    dist = []
    for idx in range(len(wind1)):
        cos = cos_sim(wind0[idx0], wind1[idx])
        arr = np.append(cos, wind1[idx][0])
        dist.append(arr)
    mean = pd.DataFrame(dist).sort_values(by= 0, ascending = False)[:10][1].mean()
    mean
    wind0[idx0][0] = mean
    return wind0[idx0][0]


In [21]:
# 함수 적용
cos = []
for i in range(len(wind0)):
    cos.append(cos_mean(i))

In [25]:
train.windspeed[train["windspeed"] == 0] = cos

<ipython-input-25-be862960e2d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.windspeed[train["windspeed"] == 0] = cos


In [26]:
train.to_csv("train_cos.csv")